### Importación de librerías

In [1]:
from selenium import webdriver # Webscrapping bot
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromiumOptions

from selenium.webdriver.common.by import By

In [2]:
import logging # Para generar logs
from logging.handlers import TimedRotatingFileHandler
from logging import Formatter

import datetime
import os
import time

In [3]:
import pandas as pd #Manejo de dataframes

### Variables a modificar para adaptar el código

In [4]:
urlToScrap ="https://images.google.com/"
deleteOldLogs = True

pathToDF = "../Outputs/"
fileToDF = "ProdsToScrap.csv"

webdriverToUse = "chromium"

### Pequeñas funciones de apoyo

In [5]:
# Esta función se ha creado para mejorar comprensión de código en la configuración de logs

def UTCFormatter(logFormatter):
    '''
    Recibe un formatter de logeo
    Devuelve el horario a tiempo GMT
    '''
    logFormatter.converter = time.gmtime
    return logFormatter

### Configuración de logs

In [6]:
# Se inicia el proceso de registro de logs a nivel de INFO.
logger = logging.getLogger('ScrapLog')
logger.setLevel(logging.INFO)

# Variables que determinan apartados posteriores
timestamp = datetime.datetime.utcnow().strftime('%Y%m%d_%H-%M-%S')
filename=f'ScrapImages{timestamp}.log'
formatter = logging.Formatter('[%(asctime)s] %(name)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')

In [7]:
'''
Indican como se debe crear el archivo de log
Si "deleteOldLogs" es True, sólo se conservará el último archivo de log
'''

if deleteOldLogs ==True:
    listFilesinCWD = os.listdir(os.getcwd())
    for element in listFilesinCWD:
        if element.endswith(".log"):
            os.remove(os.path.join(os.getcwd(), element))

fileHandler = logging.FileHandler(filename=filename)
logging.Formatter.converter = time.gmtime

fileHandler.setLevel(logging.INFO)
fileHandler.setFormatter(UTCFormatter(formatter))
logger.addHandler(fileHandler)

### Importación de datos

In [8]:
df = pd.read_csv(f"{pathToDF}{fileToDF}")
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [9]:
print(df.shape)
df.head(2)

(19778, 3)


,id,product_id,name
0,0,5645,Weleda Hombre Crema Hidratante
1,1,28743,Gynea Gestagyn Men


### Lógica del Scrapping

In [25]:
def ScrapFunction(prodToScrap, urlToScrap, webdriverUsed="chromium"):

    logger.info(f"Started with: {prodToScrap}")

    if webdriverToUse != "firefox":
        opts = ChromiumOptions()
        opts.add_argument("--no-sandbox")
        opts.add_argument("--incognito")
        opts.add_argument("start-maximized")
        opts.add_argument("window-size=1920,1080")
        opts.add_argument("--headless")
        opts.add_experimental_option("excludeSwitches", ["enable-automation"])
        opts.add_experimental_option('useAutomationExtension', False)
        driver = webdriver.Chrome(options=opts)
    else:
        opts = FirefoxOptions()
        opts.add_argument("--headless")
        opts.add_argument("--no-sandbox")
        opts.add_argument("--incognito")
        opts.add_argument("window-size=1400,600")
        driver = webdriver.Firefox(options=opts)


    driver.set_page_load_timeout(30)
    driver.set_window_size(1920, 1080)

    driver.get(urlToScrap)
    driver.delete_all_cookies()

    acceptCookie = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[3]/span/div/div/div/div[3]/button[2]/div")
    acceptCookie.click()
    driver.implicitly_wait(8)

    selectImageBox = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
    driver.implicitly_wait(8)
    selectImageBox.send_keys(prodToScrap)
    selectImageBox.send_keys(Keys.ENTER)

    selectImage = driver.find_element(By.XPATH, "/html/body/div[2]/c-wiz/div[4]/div[1]/div/div/div/div[1]/div[1]/span/div[1]/div[1]/div[1]/a[1]/div[1]/img")
    selectImage.click()

    driver.implicitly_wait(8)
    driver.refresh()
    driver.implicitly_wait(8)

    urlImage = driver.find_element(By.XPATH, "/html/body/div[2]/c-wiz/div[4]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div/a/img").get_attribute("src")

    print(urlImage)

    logger.info(f"Scrapped: {urlImage}")
    
    driver.close()

    return urlImage

In [26]:
print("Starting Webscrapping!")

for position, element in enumerate(df["name"].iloc[:10].tolist()):
    urlScrapped = ScrapFunction(element,urlToScrap,webdriverUsed=webdriverToUse)
    df.loc[df.index[position], 'url'] = urlScrapped

Starting Webscrapping!


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/c-wiz/div[4]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div/a/img"}
  (Session info: headless chrome=100.0.4896.75)
Stacktrace:
#0 0x559748940ad3 <unknown>
#1 0x5597486a0568 <unknown>
#2 0x5597486d6c46 <unknown>
#3 0x5597486d6e01 <unknown>
#4 0x559748709a64 <unknown>
#5 0x5597486f461d <unknown>
#6 0x559748707824 <unknown>
#7 0x5597486f44e3 <unknown>
#8 0x5597486c9d1a <unknown>
#9 0x5597486cae75 <unknown>
#10 0x55974896eefd <unknown>
#11 0x55974898819b <unknown>
#12 0x559748970c65 <unknown>
#13 0x559748988ec8 <unknown>
#14 0x559748964360 <unknown>
#15 0x5597489a4a68 <unknown>
#16 0x5597489a4be8 <unknown>
#17 0x5597489be7fd <unknown>
#18 0x7fd9ab24cea7 <unknown>


In [16]:
df.head()

,id,product_id,name,url
0,0,5645,Weleda Hombre Crema Hidratante,https://weledaint-prod.global.ssl.fastly.net/b...
1,1,28743,Gynea Gestagyn Men,https://www.gynea.com/wp-content/uploads/2018/...
2,2,68986,Endocare Tensage Ampollas,https://www.cantabrialabs.es/wp-content/upload...
3,3,9692,Lacer Colutorio Fluor+Xilitol Sabor Fresa,https://statics.promofarma.com/static/promofar...
4,4,81921,Age Protect Sérum Intensivo Multiacción Uriage,https://cdns3-2.primor.eu/62309-thickbox/age-p...


In [17]:
df.to_csv("Test1.csv")